In [28]:
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from agent.model import TD3_Actor
from environment.env import TradingEnv
from pykalman import KalmanFilter
import matplotlib.pyplot as plt

In [62]:
from agent.latent_space.LFSS import PretrainedAEModel

In [63]:
ae = PretrainedAEModel()

In [64]:
ae.train()

-------------Initialize Autoencoder-------------
Epoch: 0 | Loss: 0.964616758208121
Epoch: 1 | Loss: 0.8992378250245125
Epoch: 2 | Loss: 0.8423859611634286
Epoch: 3 | Loss: 0.7877804867682918
Epoch: 4 | Loss: 0.7367671881952593
Epoch: 5 | Loss: 0.6873376715567804
Epoch: 6 | Loss: 0.6407783858237728
Epoch: 7 | Loss: 0.5967896946014897
Epoch: 8 | Loss: 0.5550714356283988
Epoch: 9 | Loss: 0.5157995868113733
Epoch: 10 | Loss: 0.4784329918123061
Epoch: 11 | Loss: 0.4433646923111331
Epoch: 12 | Loss: 0.4104060655640018
Epoch: 13 | Loss: 0.3793143572345857
Epoch: 14 | Loss: 0.35011956864787686
Epoch: 15 | Loss: 0.32264162551972175
Epoch: 16 | Loss: 0.2968511490091201
Epoch: 17 | Loss: 0.27268480725826755
Epoch: 18 | Loss: 0.25015486728760505
Epoch: 19 | Loss: 0.22908889814730612
Epoch: 20 | Loss: 0.20947144300706924
-------------Finish pretraining the model!-------------
Total Training Epochs: 20
